In [1]:
import os
import pydot
import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import InceptionV3

In [2]:
model = load_model(r'./pre_train_file/Classification_model.hdf5')

In [2]:
K.clear_session()

TypeError: clear_session() got an unexpected keyword argument 'graph'

In [3]:
sess = tf.Session(graph=tf.Graph())
K.set_session(sess)
K.set_learning_phase(0)

In [4]:
# model = InceptionV3(weights='imagenet', input_shape=(256, 256, 3))
model = load_model(r'./pre_train_file/Classification_model.hdf5')

RuntimeError: The Session graph is empty.  Add operations to the graph before calling run().

In [4]:
gd = sess.graph.as_graph_def()
print(len(gd.node), 'Nodes')
#gd.node[:2]

4964 Nodes


In [12]:
gd.node[0]

name: "input_1"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 256
      }
      dim {
        size: 256
      }
      dim {
        size: 3
      }
    }
  }
}

In [5]:
# Replace Switch nodes with Identity
nodes_with_switch_op = [x for x in gd.node if x.op.lower().find('switch') != -1]
#nodes_with_switch_op[0]

In [6]:
nodes = {}
for node in gd.node:
    nodes[node.name] = node

In [7]:
print(len(nodes_with_switch_op))
[(n.name, [i for i in n.input]) for n in nodes_with_switch_op[:10]]

0


[]

In [8]:
nodes['batch_normalization/cond/ReadVariableOp/Switch']

name: "batch_normalization/cond/ReadVariableOp/Switch"
op: "Switch"
input: "batch_normalization/beta"
input: "batch_normalization/cond/pred_id"
attr {
  key: "T"
  value {
    type: DT_RESOURCE
  }
}
attr {
  key: "_class"
  value {
    list {
      s: "loc:@batch_normalization/beta"
    }
  }
}

In [9]:
nodes['batch_normalization/beta']

name: "batch_normalization/beta"
op: "VarHandleOp"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@batch_normalization/beta"
    }
  }
}
attr {
  key: "container"
  value {
    s: ""
  }
}
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: 32
      }
    }
  }
}
attr {
  key: "shared_name"
  value {
    s: "batch_normalization/beta"
  }
}

In [10]:
# nodes to reroute switch input
for n in [x for x in gd.node]:
    ints = [i for i in n.input]
    endswith1 = [ii for ii in ints if ii.endswith(':1')] #and 'Switch' in ii]
    if len(endswith1) > 0:
        for index, inn in enumerate(n.input):
            if inn in endswith1:
                new_input = inn[:-2]
                n.input.remove(inn)
                n.input.insert(index, new_input)

In [11]:
for n in nodes_with_switch_op:
    n.op = 'Identity'
    n.input.pop()

In [15]:
nodes_with_switch_op

[name: "batch_normalization/cond/Switch"
 op: "Identity"
 input: "batch_normalization/keras_learning_phase"
 attr {
   key: "T"
   value {
     type: DT_BOOL
   }
 }, name: "batch_normalization/cond/ReadVariableOp/Switch"
 op: "Identity"
 input: "batch_normalization/beta"
 attr {
   key: "T"
   value {
     type: DT_RESOURCE
   }
 }
 attr {
   key: "_class"
   value {
     list {
       s: "loc:@batch_normalization/beta"
     }
   }
 }, name: "batch_normalization/cond/FusedBatchNorm/Switch"
 op: "Identity"
 input: "conv2d/Conv2D"
 attr {
   key: "T"
   value {
     type: DT_FLOAT
   }
 }
 attr {
   key: "_class"
   value {
     list {
       s: "loc:@conv2d/Conv2D"
     }
   }
 }, name: "batch_normalization/cond/FusedBatchNorm/Switch_1"
 op: "Identity"
 input: "batch_normalization/Const"
 attr {
   key: "T"
   value {
     type: DT_FLOAT
   }
 }
 attr {
   key: "_class"
   value {
     list {
       s: "loc:@batch_normalization/Const"
     }
   }
 }, name: "batch_normalization/cond/Rea

In [10]:
[(n.name, [i for i in n.input]) for n in gd.node if n.name.lower().find('output_node') != -1]

[]

In [9]:
model.output

<tf.Tensor 'final_output/Softmax:0' shape=(?, 2) dtype=float32>

In [11]:
# nb_classes = 2
# prefix_output_node_names_of_final_network = 'output_node'
# pred = [None]*nb_classes
# pred_node_names = [None]*nb_classes
# for i in range(nb_classes):
#     pred_node_names[i] = prefix_output_node_names_of_final_network+str(i)
#     pred[i] = tf.identity(model.output[i], name=pred_node_names[i])
output_fld = 'pre_train_model/'
if not os.path.isdir(output_fld):
    os.mkdir(output_fld)
output_graph_name = 'test.pb'

constant_graph = graph_util.convert_variables_to_constants(sess, gd, ['final_output/Softmax'])
graph_io.write_graph(constant_graph, output_fld, output_graph_name, as_text=False)

INFO:tensorflow:Froze 378 variables.
INFO:tensorflow:Converted 378 variables to const ops.


'pre_train_model/test.pb'

In [7]:
filename = r'./pre_train_/Classification_model.pb'
g = tf.GraphDef()
g.ParseFromString(open(filename, 'rb').read())
print()
print(filename)
print("=======================INPUT=========================")
print([n for n in g.node if n.name.find('input') != -1])
print("=======================OUTPUT========================")
print([n for n in g.node if n.name.find('output') != -1])
print("===================KERAS_LEARNING=====================")
print([n for n in g.node if n.name.find('keras_learning_phase') != -1])
print("======================================================")


./pre_train_file/Classification_model.pb
=======================INPUT=========================
[name: "batch_normalization/keras_learning_phase/input"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_BOOL
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_BOOL
      tensor_shape {
      }
      bool_val: false
    }
  }
}
, name: "input_1_1"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 256
      }
      dim {
        size: 256
      }
      dim {
        size: 3
      }
    }
  }
}
]
=======================OUTPUT========================
[name: "final_output_2/kernel"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
        dim {
          size: 2048
        }
        dim {
          size: 2
        }
      }
      tensor

In [9]:
model = load_model(r'./pre_train_file/Classification_model.hdf5')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [31]:
model = inception_model

In [32]:
nb_classes = 2
prefix_output_node_names_of_final_network = 'output_node'
pred = [None]*nb_classes
pred_node_names = [None]*nb_classes
for i in range(nb_classes):
    pred_node_names[i] = prefix_output_node_names_of_final_network+str(i)
    pred[i] = tf.identity(model.output[i], name=pred_node_names[i])
output_fld = 'pre_train_model/'
if not os.path.isdir(output_fld):
    os.mkdir(output_fld)
output_graph_name = 'test.pb'

constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
graph_io.write_graph(constant_graph, output_fld, output_graph_name, as_text=False)

FailedPreconditionError: Error while reading resource variable batch_normalization_217/moving_mean from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/batch_normalization_217/moving_mean)
	 [[{{node batch_normalization_217/moving_mean/Read/ReadVariableOp}} = ReadVariableOp[dtype=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](batch_normalization_217/moving_mean)]]

Caused by op 'batch_normalization_217/moving_mean/Read/ReadVariableOp', defined at:
  File "C:\Users\46176\Anaconda3\Lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\46176\Anaconda3\Lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\46176\documents\files\study\ec601\smart_vision_app_generator\svag_env\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\46176\documents\f

In [8]:
filename = r'pre_train_model/test.pb'
g = tf.GraphDef()
g.ParseFromString(open(filename, 'rb').read())
print()
print(filename)
print("=======================INPUT=========================")
print([n for n in g.node if n.name.find('input') != -1])
print("=======================OUTPUT========================")
print([n for n in g.node if n.name.find('output') != -1])
print("===================KERAS_LEARNING=====================")
print([n for n in g.node if n.name.find('keras_learning_phase') != -1])
print("======================================================")


pre_train_model/test.pb
=======================INPUT=========================
[name: "input_1"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 256
      }
      dim {
        size: 256
      }
      dim {
        size: 3
      }
    }
  }
}
, name: "batch_normalization/keras_learning_phase/input"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_BOOL
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_BOOL
      tensor_shape {
      }
      bool_val: false
    }
  }
}
]
=======================OUTPUT========================
[name: "final_output/kernel"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
        dim {
          size: 2048
        }
        dim {
          size: 2
        }
      }
      tensor_content: "4B?\274\23

In [23]:
from tensorflow.keras.applications import InceptionV3

In [24]:
inception_model = InceptionV3(weights='imagenet', input_shape=(256, 256, 3))

In [25]:
inception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [26]:
model = inception_model

In [2]:
file_dir = r'C:\Users\46176\Documents\files\study\EC601\Smart_Vision_App_Generator\test\userId\ProjectName\datasetName'

In [3]:
IMAGE_SIZE = [256, 256]
CHANNELS = 3
SEGMENTATION_IMAGE_BATCH_SIZE = 100
CLASSIFY_IMAGE_BATCH_SIZE = 10
BOTTLENECK_BATCH_SIZE = 500
VALIDATION_PERCENTAGE = 0.3
CLASSIFY_LEARNING_RATE = 0.0005
CLASSIFY_EPOCH = 20

IMAGE_FOLDER = 'images'
LABEL_FOLDER = 'labels'
LABEL_INFO_FILE = 'labels_info.txt'
BOTTLENECK_FOLDER = 'bottlenecks'
TRAINED_MODEL_FOLDER = 'models'
INCEPTION_V3_FILE = r'.\pre_train_file\inceptionV3_bottleneck.hdf5'

In [4]:
gfile.Exists(INCEPTION_V3_FILE)

True

In [5]:
saved_models_path = os.path.join(file_dir, TRAINED_MODEL_FOLDER)
images_dir = os.path.join(file_dir, IMAGE_FOLDER)
label_file = os.path.join(file_dir, LABEL_INFO_FILE)
bottleneck = os.path.join(file_dir, BOTTLENECK_FOLDER)

image_dp = ImageDataPipeline(images_dir, label_file, image_size=IMAGE_SIZE)
generator = InceptionModelGenerator(INCEPTION_V3_FILE,
                                    image_dp.labels_classes,
                                    (IMAGE_SIZE[0], IMAGE_SIZE[1], CHANNELS))
bottleneck_dp = BottleneckDataPipeline(bottleneck, label_file)

In [9]:
train.get_and_cache_bottlenecks(file_dir, generator, image_dp)

Looking for images in 'n0'
Looking for images in 'n1'
Looking for images in 'n2'
Looking for images in 'n3'
Looking for images in 'n4'
Looking for images in 'n5'
Looking for images in 'n6'
Looking for images in 'n7'
Looking for images in 'n8'
Looking for images in 'n9'
already output 0 images' bottlenecks
already output 100 images' bottlenecks
already output 200 images' bottlenecks
already output 300 images' bottlenecks
already output 400 images' bottlenecks
already output 500 images' bottlenecks
already output 600 images' bottlenecks
already output 700 images' bottlenecks
already output 800 images' bottlenecks
already output 900 images' bottlenecks
already output 1000 images' bottlenecks


In [7]:
weights_file = train.train_one_layer_model(file_dir, generator, bottleneck_dp)

Looking for images in 'n0'
Looking for images in 'n1'
Looking for images in 'n2'
Looking for images in 'n3'
Looking for images in 'n4'
Looking for images in 'n5'
Looking for images in 'n6'
Looking for images in 'n7'
Looking for images in 'n8'
Looking for images in 'n9'
Epoch 1/20
1/2 [==============>...............] - ETA: 1s - loss: 2.1292 - acc: 0.3200
Epoch 00001: val_acc improved from -inf to 0.47200, saving model to C:\Users\46176\Documents\files\study\EC601\Smart_Vision_App_Generator\test\userId\ProjectName\datasetName\models\one_layer_weights.hdf5
2/2 [==============================] - 3s 1s/step - loss: 2.0668 - acc: 0.3650 - val_loss: 1.8723 - val_acc: 0.4720
Epoch 2/20
1/2 [==============>...............] - ETA: 0s - loss: 1.8242 - acc: 0.5320
Epoch 00002: val_acc improved from 0.47200 to 0.66000, saving model to C:\Users\46176\Documents\files\study\EC601\Smart_Vision_App_Generator\test\userId\ProjectName\datasetName\models\one_layer_weights.hdf5
2/2 [========================